<div style="text-align: center;">
    <img src="./images/logo_CESI.jpg" width="500">
</div>

# Livrable 1

## Contexte du Projet

Ce projet vise à répondre à l'appel à manifestation d'intérêt lancé par l'ADEME (Agence de l'Environnement et de la Maîtrise de l'Énergie) pour la réalisation de démonstrateurs et d'expérimentations de nouvelles solutions de mobilité durable adaptées à différents types de territoires.

L'équipe de CesiCDP, en collaboration avec plusieurs partenaires, s'est déjà intéressée à la mobilité multimodale intelligente et souhaite développer une méthode basée sur la recherche opérationnelle pour résoudre le problème de la gestion de tournées de livraison. L'objectif est de calculer une tournée optimisée sur un réseau routier reliant un sous-ensemble de villes, en minimisant la durée totale de la tournée tout en tenant compte du trafic prévu sur chaque axe pour les différentes tranches horaires.

Le projet comporte une version de base du problème où le modèle et le code en Python doivent être développés pour résoudre des instances de taille importante. De plus, une étude statistique du comportement expérimental de l'algorithme doit être réalisée.

Dans un second temps, des contraintes supplémentaires peuvent être intégrées. Ces dernières incluent des fenêtres de temps de livraison, l'interdiction de livrer en dehors de ces fenêtres, la possibilité d'attendre sur place l'ouverture des fenêtres, l'utilisation de plusieurs camions pour effectuer les livraisons avec des contraintes de capacité et d'encombrement, des points de collecte spécifiques pour chaque objet, la variation du temps de parcours des axes en fonction du trafic, etc.

Le projet est organisé en plusieurs étapes, dont la modélisation formelle, la conception algorithmique et l'implémentation, l'étude expérimentale et la présentation des résultats à l'équipe avant la remise des livrables à l'ADEME.

Le premier livrable de modélisation contient une étude décrivant le problème, sa formalisation, les contraintes supplémentaires traitées et une analyse théorique de sa complexité.

## Membre du groupe

<img src="./images/team.png" width="1000">

# Optimisation de Tournées de Livraison
## Contexte
L'ADEME a lancé un appel à manifestation d'intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité. CesiCDP, en collaboration avec ses partenaires, propose une solution pour optimiser les tournées de livraison.

## Description du Problème de Base
Le problème consiste à calculer une tournée de livraison sur un réseau routier reliant plusieurs villes et revenant au point de départ, en minimisant la durée totale de la tournée. Ce problème est connu sous le nom de Problème du Voyageur de Commerce (TSP).


## Reformulation Formelle du Problème
### Variables de Décision
- \( x_{ij} \): Indicateur binaire (1 si l'arc de la ville \( i \) à la ville \( j \) est utilisé, 0 sinon)
- \( t_i \): Temps d'arrivée à la ville \( i \)

### Fonction Objectif
Minimiser la durée totale de la tournée:
\[
\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} x_{ij}
\]
où \( c_{ij} \) est le coût (temps de trajet) entre les villes \( i \) et \( j \).

### Contraintes
1. Chaque ville doit être visitée une fois:
\[
\sum_{j=1}^{n} x_{ij} = 1 \quad \forall i
\]
2. Chaque ville doit être quittée une fois:
\[
\sum_{i=1}^{n} x_{ij} = 1 \quad \forall j
\]
3. Temps de trajet:
\[
t_i + c_{ij} - t_j \leq M (1 - x_{ij}) \quad \forall i, j
\]
4. Variables binaires:
\[
x_{ij} \in \{0, 1\}
\]


## Propriétés Théoriques du Problème
Le problème de base est un problème NP-difficile, similaire au Problème du Voyageur de Commerce (TSP). La complexité théorique du problème est confirmée par sa réduction au TSP, ce qui indique qu'il n'existe pas d'algorithme en temps polynomial pour le résoudre exactement.

### Comparaison avec d'autres Problèmes
- **TSP**: Chaque ville est visitée une fois avec un retour au point de départ.
- **VRP**: Similaire au TSP mais avec des véhicules multiples et des contraintes de capacité.


## Contraintes Supplémentaires
### Fenêtres de Temps de Livraison
- Interdiction de livrer hors de la fenêtre
- Possibilité d'attendre sur place

### Variation du Temps de Parcours
Le temps de parcours d'une arête varie au cours du temps pour représenter la variation du trafic. Cela ajoute une dimension temporelle au coût \( c_{ij} \).


## Références Bibliographiques
- Exemple d'article 1
- Exemple d'article 2
- Ouvrages scientifiques sur le TSP et le VRP


In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 29, 20, 21],
        [29, 0, 15, 17],
        [20, 15, 0, 28],
        [21, 17, 28, 0],
    ]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def main():
    data = create_data_model()
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Variables
    num_locations = len(data['distance_matrix'])
    x = {}
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                x[i, j] = solver.BoolVar(f'x[{i},{j}]')
    
    # Constraints
    for i in range(num_locations):
        solver.Add(sum(x[i, j] for j in range(num_locations) if i != j) == 1)
        solver.Add(sum(x[j, i] for j in range(num_locations) if i != j) == 1)
    
    # Objective
    objective = solver.Objective()
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                objective.SetCoefficient(x[i, j], data['distance_matrix'][i][j])
    objective.SetMinimization()
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Total cost = ', objective.Value())
        for i in range(num_locations):
            for j in range(num_locations):
                if i != j:
                    if x[i, j].solution_value() > 0:
                        print(f'Location {i} to {j}')
    else:
        print('No solution found.')

if __name__ == '__main__':
    main()
